Demonstrate functions in Scan

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
soc = TopSoc()

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)

In [ ]:
fMixer = soc.fsIn/2/2
soc.set_mixer(fMixer)

In [ ]:
fTone = fMixer + soc.fsOut/np.pi + 0.523
#fTone = 500.3
print("fTone =",fTone)
inCh,inOffset = soc.inFreq2chOffset(fTone)
outCh,outOffset = soc.outFreq2chOffset(fTone)
print("fTone = %.6f MHz"%fTone)
print("  inCh=%4d    inOffset=%+.6f"%(inCh, inOffset))
print(" outCh=%4d   outOffset=%+.6f"%(outCh, outOffset))

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
tones = np.array([fTone])
amplitudes = 0.9*np.ones(len(tones))/len(tones)
fis = 0*np.ones(len(tones))
scan.setTones(tones, amplitudes, fis)
decimation=2
scan.prepRead(decimation)


In [ ]:
xs = scan.read()
xi = xs[0,:,0]
xq = xs[0,:,1]
plt.plot(xi)
plt.plot(xq)

In [ ]:
xs = scan.read()
xi = xs[0,:,0]
xq = xs[0,:,1]
plt.plot(xi)
plt.plot(xq)

In [ ]:
xs = scan.read()
xi = xs[0,:,0]
xq = xs[0,:,1]
plt.plot(xi)
plt.plot(xq)